In [ ]:
import muda
import glob
import os
import tqdm

folder = "fold3"
folder_path = f"data/UrbanSound8K/augmented_data/train/{folder}"
original_jams_file_path = f"{folder_path}/original/jams"
audio_files_glob = f"{folder_path}/original/audio/*.wav"

deformers_with_name = []

# Setup background deformer
background_noise_deformation_name = "bgnoise"
background_audio_files_glob = "data/UrbanSound8K/background_noise/*.wav"
background_deformer = muda.deformers.BackgroundNoise(files=glob.glob(background_audio_files_glob))

deformers_with_name.append((background_noise_deformation_name, background_deformer))

# Setup time stretching deformer
time_stretch_deformation_name = "stretch"
time_stretch_deformer = muda.Union(steps=[(f"time-stretch-{i}", muda.deformers.TimeStretch(rate=stretch_factor)) for i, stretch_factor in enumerate([0.81, 0.93, 1.07, 1.23])])

deformers_with_name.append((time_stretch_deformation_name, time_stretch_deformer))

# Setup pitch shifting 1
pitch_shift_1_name = "pitch1"
pitch_shift_1_deformer = muda.Union(steps=[(f"pitch-shift-1-{i}", muda.deformers.PitchShift(n_semitones=semitone_shift_factor)) for i, semitone_shift_factor in enumerate([-2, -1, 1, 2])])

deformers_with_name.append((pitch_shift_1_name, pitch_shift_1_deformer))

# Setup pitch shifting 2
pitch_shift_2_name = "pitch2"
pitch_shift_2_deformer = muda.Union(steps=[(f"pitch-shift-2-{i}", muda.deformers.PitchShift(n_semitones=semitone_shift_factor)) for i, semitone_shift_factor in enumerate([-3.5, -2.5, 2.5, 3.5])])

deformers_with_name.append((pitch_shift_2_name, pitch_shift_2_deformer))

# Setup Dynamic Range compression
dynamic_range_compression_name = "drc"
dynamic_range_compression_deformer = muda.deformers.DynamicRangeCompression(preset=["music standard", "film standard", "speech", "radio"])

deformers_with_name.append((dynamic_range_compression_name, dynamic_range_compression_deformer))

audio_files_paths = sorted(glob.glob(audio_files_glob))
for deformer_name, deformer in deformers_with_name:
    print(f"Processing files with deformer: {deformer_name} ⚙️")

    for audio_file_path in tqdm.tqdm(audio_files_paths):
        deformed_audio_folder_path = f"{folder_path}/{deformer_name}/audio"
        deformed_jams_folder_path = f"{folder_path}/{deformer_name}/jams"
        os.makedirs(deformed_audio_folder_path, exist_ok=True)
        os.makedirs(deformed_jams_folder_path, exist_ok=True)
        file_name = os.path.basename(audio_file_path)[:-4]
        original_jam = muda.load_jam_audio(f"{original_jams_file_path}/{file_name}.jams", audio_file=audio_file_path)
        for i, deformed_jam in enumerate(deformer.transform(original_jam)):
            muda.save(f"{deformed_audio_folder_path}/{file_name}_{deformer_name}{i}.wav", f"{deformed_jams_folder_path}/{file_name}_{deformer_name}{i}.jams", deformed_jam)

    print(f"Done processing files with deformer: {deformer_name} ✅")

In [32]:
# Create validation dataset from augmented dataset
from random import Random
import os
import glob
import tqdm

validation_split = 0.2
rng = Random(42)
fold = "fold3"
validation_dataset = f"data/UrbanSound8K/augmented_data/validation/{fold}"
train_dataset = f"data/UrbanSound8K/augmented_data/train/{fold}"

sample_names = list(map(lambda sample_path: sample_path.split("/")[-1][:-5], glob.glob(f"{train_dataset}/original/jams/*.jams")))
num_validation_samples = int(len(sample_names) * validation_split)

for i in tqdm.tqdm(range(num_validation_samples)):
    rand_sample_index = rng.randint(0, len(sample_names) - 1)
    sample_name_to_move = sample_names.pop(rand_sample_index)

    for modification_name in filter(lambda mod: not mod.startswith("."), os.listdir(train_dataset)):
        os.makedirs(f"{validation_dataset}/{modification_name}/jams", exist_ok=True)
        os.makedirs(f"{validation_dataset}/{modification_name}/audio", exist_ok=True)

        # Each modification may have created multiple copies of the original
        jams_file_glob = f"{train_dataset}/{modification_name}/jams/{sample_name_to_move}*.jams"
        for jams_file_path in glob.glob(jams_file_glob):
            os.rename(jams_file_path, jams_file_path.replace("train", "validation")) # move jams file
        audio_file_glob = f"{train_dataset}/{modification_name}/audio/{sample_name_to_move}*.wav"
        for audio_file_path in glob.glob(audio_file_glob):
            os.rename(audio_file_path, audio_file_path.replace("train", "validation")) # move wav file

100%|██████████| 185/185 [00:08<00:00, 21.65it/s]


## Folds processed

[x] Fold 1
[x] Fold 2
[x] Fold 3
[ ] Fold 4
[ ] Fold 5
[ ] Fold 6
[ ] Fold 7
[ ] Fold 8
[ ] Fold 9
[ ] Fold 10

In [ ]:
# Generate command to undo the operations
validation_dataset = f"data/UrbanSound8K/augmented_data/validation/{fold}"
train_dataset = f"data/UrbanSound8K/augmented_data/train/{fold}"

for root, subdirs, files in os.walk(validation_dataset):
    if len(files) > 0:
        print(f"mv {root}/* {root.replace('validation', 'train')}")

## Convert Audio Files to Images

In [9]:
from jams_to_image_data_generator import JamsToImageDataGenerator
import numpy as np
import os
import tqdm

fold = "fold1"
dataset = "validation"
train_dataset = f"data/UrbanSound8K/augmented_data/{dataset}/{fold}"

image_generator = JamsToImageDataGenerator(train_dataset)
images, labels = image_generator[0]
for batch_index in tqdm.tqdm(range(1, len(image_generator))):
    batch_images, batch_labels = image_generator[batch_index]
    images = np.concatenate((images, batch_images))
    labels = np.concatenate((labels, batch_labels))

images_path = f"data/UrbanSound8K/augmented_data_images/{dataset}/{fold}"
os.makedirs(images_path, exist_ok=True)
np.save(f"{images_path}/images.npy", images)
np.save(f"{images_path}/labels.npy", labels)

100%|██████████| 64/64 [00:32<00:00,  1.95it/s]
